# No Knowledge
* **Event:** HackTX CTF
* **Problem Type:** Cryptography
* **Point Value / Difficulty:**
* **(Optional) Tools Required / Used:**


## Background Information
- https://en.wikipedia.org/wiki/Modular_arithmetic
- https://en.wikipedia.org/wiki/Discrete_logarithm
- https://en.wikipedia.org/wiki/Zero-knowledge_proof#Discrete_log_of_a_given_value

## Solution Idea

We are given a file called `zkip.py` which corresponds to the code that's running on the server and `willsmith.txt` which contains Will Smith's public parameters.
Victor the Verifier will only grant the flag if he is convinced that we are Will Smith by winning his game.
But we can trick him because his questions are not random at all, they just alternate!

I copied the two files below for convenience, run the cell if you want to try solving.

In [ ]:
g = 2
p = 9375119790450996218800455883451345336695855790791241020058042501987900920600528738223355476522262581439097496402411226460716471247425673326484948646632061
y = 5954245469284513610947819945924138072213664706770638382722170958711863430250348082109154748988457221560891498741505152926581333481648986146668353438353092

In [ ]:
def verify1(g, p, C):
    print("Send me r: ")
    r = input()
    if r.isnumeric():
        r = int(r)
        return C == pow(g, r, p)
    return false


def verify2(g, p, y, C):
    print("Send me (x + r) mod (p - 1): ")
    xr = input()
    if xr.isnumeric():
        xr = int(xr)
        return (C*y % p) == pow(g, xr, p)
    return false


def proof_round(g, p, y, n):
    print("Compute a random number r and send me g^r mod p: ")
    C = input()
    if C.isnumeric():
        C = int(C)
        if n % 2 == 0:
            return verify1(g, p, C)
        else:
            return verify2(g, p, y, C)

def main():

    # WILL SMITH's public parameters
    g = 2
    p = 9375119790450996218800455883451345336695855790791241020058042501987900920600528738223355476522262581439097496402411226460716471247425673326484948646632061
    y = 5954245469284513610947819945924138072213664706770638382722170958711863430250348082109154748988457221560891498741505152926581333481648986146668353438353092

    print("Are you actually WILL SMITH?? Prove it to me by playing my stupid game!")
    print("Don't worry Mr. Smith, if you play my game I will have ZERO KNOWLEDGE of your secret password x :^)")

    flag = True

    for i in range(1, 101):
        print("ROUND " + str(i))
        if proof_round(g, p, y, i):
            print("OK NEXT ROUND")
        else:
            print("You are not WILL SMITH!")
            flag = False
            break

    if flag:
        with open('FLAG.txt', 'r') as f:
            print(f.readlines()[0].strip())

main()

## Solution

Zero Knowledge proofs work in the following way:
1. Peggy the Prover is trying to convince Victor the Verifier that she knows some secret value $x$.
2. Peggy doesn't want to leak any information about $x$, otherwise the problem would be really easy (just send Victor x).
3. Victor is paranoid and will ask Peggy a bunch of questions related to $x$ in some way without leaking any information about $x$.
4. Once Victor has asked enough questions he will become convinced that Peggy actually knows the value $x$.

The scenario implemented by the server is exactly the one described here https://en.wikipedia.org/wiki/Zero-knowledge_proof#Discrete_log_of_a_given_value

Since Victor's questions are alternating we can easily trick him in the following way:

1. He first asks us for $C = g^r \pmod N$ for some random value $r$ (there is nothing stopping us from choosing $r = 0$).
2. Before we send him the $C$ value we should figure out which question he's going to ask.
3. If he asks `verify1` then we should send him $1$ followed by $0$.
    - This is because `C == pow(g, r, p)` will evaluate to true
4. If he asks `verify2` then we should send him $y^{-1}$ followed by $0$.
    - This is because `(C*y % p) == pow(g, xr, p)` will evaluate to true